In [ ]:
import graph_generator as gg
import obm_dp as dp
import numpy as np
from torch_geometric.loader import DataLoader
from torch_geometric.nn import NNConv, TransformerConv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from util import diff
from tqdm import trange
import copy
from scipy.optimize import linear_sum_assignment

%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
print("PyTorch has version {}".format(torch.__version__))
print('Using device:', device)

In [ ]:
m = 10; n = 6; train_num = 100; test_num = 50

er_config = {
    'graph_type': 'ER',
    'p': 1,
    'weighted': True
}
ba_config = {
    'graph_type': 'BA',
    'ba_param': 4,
    'weighted': True
}
geom_config = {
    'graph_type': 'GEOM',
    'threshold': 0.2,
    'scaling': 1 / np.sqrt(2)
}

train_dataset = DataLoader(
    [
        *gg.generate_examples(train_num, m, n, [0.8] * m, **er_config),
        *gg.generate_examples(train_num, m, n, [0.8] * m, **ba_config),
        *gg.generate_examples(train_num, m, n, [0.8] * m, **geom_config)
         
    ]
)
test_dataset = DataLoader(
    [
        *gg.generate_examples(test_num, m, n, [0.8] * m, **er_config),
        *gg.generate_examples(test_num, m, n, [0.8] * m, **ba_config),
        *gg.generate_examples(test_num, m, n, [0.8] * m, **geom_config)
         
    ]
)

In [ ]:
ex = list(test_dataset)[0]

In [ ]:
ex.hint

In [ ]:
class OBM_NNConv(torch.nn.Module):
    """
    GNN to predict node-level embeddings. Then applies a post-message passing layer to transform into the output
    dimension.
    Part of the code definition is inspired by Colab 2:
    https://colab.research.google.com/drive/1xHmpjVO-Z74NK-dH3qoUBTf-tKUPfOKW?usp=sharing

    The main model used for convolutions is NNConv from the "Dynamic Edge-Conditioned Filters in Convolutional Neural
    Networks on Graphs" <https://arxiv.org/abs/1704.02901> paper
    """

    def __init__(self, input_dim, output_dim, edge_feature_dim, args):
        """
        Initializing the GNN
        Args:
            input_dim: dimension of node features
            output_dim: output dimension required
            edge_feature_dim: dimension of the edge features
            args: object containing the rest of the GNN description, including the number of layers, dropout, ...
        """
        super(OBM_NNConv, self).__init__()

        hidden_dim = args.hidden_dim
        self.dropout = args.dropout
        aggr = args.aggr
        self.num_layers = args.num_layers
        conv_modules = [NNConv(input_dim, hidden_dim, nn.Linear(edge_feature_dim, input_dim * hidden_dim), aggr=aggr)]
        conv_modules.extend(
                [NNConv(hidden_dim, hidden_dim, nn.Linear(edge_feature_dim, hidden_dim * hidden_dim), aggr=aggr) for _ in
                 range(self.num_layers - 1)])

        self.convs = nn.ModuleList(conv_modules)
        self.regression_head = nn.Linear(hidden_dim + 1, output_dim)

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        self.regression_head.reset_parameters()

    def forward(self, x, edge_index, edge_attr, graph_features):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index, edge_attr)
            x = F.relu(x)
            x = F.dropout(x, self.dropout, self.training)
        return self.regression_head(torch.hstack((x, graph_features.view(-1, 1))))
    

class OBM_TransformerConv(torch.nn.Module):
    """
    GNN to predict node-level embeddings. Then applies a post-message passing layer to transform into the output
    dimension.
    Part of the code definition is inspired by Colab 2:
    https://colab.research.google.com/drive/1xHmpjVO-Z74NK-dH3qoUBTf-tKUPfOKW?usp=sharing

    The main model used for convolutions is NNConv from the "Dynamic Edge-Conditioned Filters in Convolutional Neural
    Networks on Graphs" <https://arxiv.org/abs/1704.02901> paper
    """

    def __init__(self, input_dim, output_dim, edge_feature_dim, args):
        """
        Initializing the GNN
        Args:
            input_dim: dimension of node features
            output_dim: output dimension required
            edge_feature_dim: dimension of the edge features
            args: object containing the rest of the GNN description, including the number of layers, dropout, ...
        """
        super(OBM_TransformerConv, self).__init__()

        hidden_dim = args.hidden_dim
        self.dropout = args.dropout
        self.num_layers = args.num_layers
        conv_modules = [TransformerConv(
            in_channels=input_dim, 
            out_channels=hidden_dim,
            heads=args.heads,
            edge_dim=edge_feature_dim)]
        conv_modules.extend(
            [
                TransformerConv(
                    in_channels=hidden_dim * args.heads, 
                    out_channels=hidden_dim,
                    heads=args.heads,
                    edge_dim=edge_feature_dim
                ) 
                for _ in range(self.num_layers - 1)
            ]
        )

        self.convs = nn.ModuleList(conv_modules)
        self.regression_head = nn.Linear(args.heads * hidden_dim + 1, output_dim)

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        self.regression_head.reset_parameters()

    def forward(self, x, edge_index, edge_attr, graph_features):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index, edge_attr)
            x = F.relu(x)
            x = F.dropout(x, self.dropout, self.training)
        return self.regression_head(torch.hstack((x, graph_features.view(-1, 1))))


In [ ]:
class MaskedMSELoss(nn.Module):

    def __init__(self):
        super(MaskedMSELoss, self).__init__()

    def forward(self, pred, value_to_go, neighbor_mask):
        """
        Computes MSE over neighbors of the arriving node.
        Args:
            pred: predicted node embeddings
            value_to_go: array of underlying value to gos
            neighbor_mask: mask for neighbors of arriving node

        Returns:
            Masked mean square error.
        """
        preds = pred[neighbor_mask].squeeze(dim=1)
        return F.mse_loss(preds, value_to_go)
    

def build_optimizer(args, params):
    """
    Builds an optimizer according to the given parameters.
    """

    weight_decay = args.weight_decay
    filter_fn = filter(lambda p: p.requires_grad, params)
    if args.opt == 'adam':
        optimizer = optim.Adam(filter_fn, lr = args.lr, weight_decay = weight_decay)
    elif args.opt == 'sgd':
        optimizer = optim.SGD(filter_fn, lr = args.lr, momentum = 0.95, weight_decay = weight_decay)
    elif args.opt == 'rmsprop':
        optimizer = optim.RMSprop(filter_fn, lr = args.lr, weight_decay = weight_decay)
    elif args.opt == 'adagrad':
        optimizer = optim.Adagrad(filter_fn, lr = args.lr, weight_decay = weight_decay)
    if args.opt_scheduler == 'none':
        return None, optimizer
    elif args.opt_scheduler == 'step':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = args.opt_decay_step, gamma = args.opt_decay_rate)
    elif args.opt_scheduler == 'cos':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = args.opt_restart)
    return scheduler, optimizer


class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [ ]:
def test(loader, test_model, loss_fn):
    test_model.eval()
    test_model.to(device)
    total_loss = 0

    for batch in loader:
        batch.to(device)
        with torch.no_grad():
            pred = test_model(batch.x, batch.edge_index, batch.edge_attr, batch.graph_features)
            loss = loss_fn(pred, batch.hint, batch.neighbors)
            total_loss += loss * batch.num_graphs

    total_loss /= len(loader.dataset)

    return total_loss

In [ ]:
NETWORKS = {
    'TransformerConv': OBM_TransformerConv,
    'NNConv': OBM_NNConv
}

def train(train_loader: DataLoader, test_loader: DataLoader, args: dict):
    """
    Trains a GNN model, periodically testing it and accumulating loss values
    Args:
        args: dictionary object containing training parameters
    """

    # Input dimension is 1 (we only have demand information for every node)
    # Edge feature dimension is 2 (capacity and cost per edge)
    # Output dimension is 1 since we predict scalar potential values for each vertex
    model = NETWORKS[args.processor](
        args.node_feature_dim,
        1,
        args.edge_feature_dim,
        args
    )
    loss_fn = MaskedMSELoss()

    _, opt = build_optimizer(args, model.parameters())
    model.to(device)

    # accumulate model performance for plotting
    train_losses = []
    test_losses = []
    best_loss = None
    best_model = None

    for epoch in trange(args.epochs, desc = "Training", unit = "Epochs"):
        total_loss = 0
        model.train()

        for batch in train_loader:
            batch.to(device)
            opt.zero_grad()
            pred = model(batch.x, batch.edge_index, batch.edge_attr, batch.graph_features)
            loss = loss_fn(pred, batch.hint, batch.neighbors)
            loss.backward()
            opt.step()

            total_loss += loss.item() * batch.num_graphs
        total_loss /= len(train_loader.dataset)
        print(total_loss)
        train_losses.append(total_loss)

        if epoch % 2 == 0:
            test_loss = test(test_loader, model, loss_fn)
            print(f'TEST LOSS: {test_loss}')
            test_losses.append(test_loss)
            if best_loss is None or test_loss < best_loss:
                best_loss = test_loss
                best_model = copy.deepcopy(model)
        else:
            test_losses.append(test_losses[-1])

    return train_losses, test_losses, best_model, best_loss

In [ ]:
# args defines the model and training hyperparameters
args = {
    'processor':     'NNConv',
    'num_layers':    3,
    'aggr':          'max',
    'batch_size':    25,
    'node_feature_dim': 4,
    'edge_feature_dim': 1,
    'hidden_dim':    128,
    'heads':         4,
    'dropout':       0.75,
    'epochs':        50,
    'opt':           'adam',
    'opt_scheduler': 'none',
    'opt_restart':   0,
    'weight_decay':  5e-3,
    'lr':            0.0001
}
args = objectview(args)

_, _, trained_model, _ = train(train_dataset, test_dataset, args)

In [ ]:
num_trials = 250
node_configs = [(48, 16), (32, 32), (16, 48)]
graph_configs = [
    {
        'graph_type': 'ER',
        'p': 0.75,
        'weighted': True
    },
    {
        'graph_type': 'BA',
        'ba_param': 4,
        'weighted': True
    },
    {
        'graph_type': 'GEOM',
        'threshold': 0.2,
        'scaling': 1 / np.sqrt(2)
    }
]

def test_model(trained_model, num_trials, node_configs, graph_configs):
    for m, n in node_configs:
        for config in graph_configs:
            print(m, n, config)
            greedy_vals = []
            learned_vals = []
            for _ in range(num_trials):
                A = gg.sample_bipartite_graph(m, n, **config)
                p = [0.8 for _ in range(m)]
                coin_flips = [np.random.binomial(1, _p) for _p in p]
                all_nodes = np.arange(n + m + 1)
                offline_nodes = frozenset(np.arange(n))
                matching = []
                value = 0
                for t in range(m):
                    if coin_flips[t]:
                        input = gg._to_pyg_test(A, p, offline_nodes, t)
                        pred = trained_model(input.x, input.edge_index, input.edge_attr, input.graph_features)
                        chosen_index = np.argmin(pred[input.neighbors].detach().numpy())
                        choice = all_nodes[input.neighbors][chosen_index]
                        if choice < n:
                            matching.append((t, choice))
                            value += A[t, choice]
                    
                        offline_nodes = diff(offline_nodes, choice)

                for t in range(m):
                    if not coin_flips[t]:
                        A[t, :] = 0
                row_ind, col_ind = linear_sum_assignment(A, maximize=True)
                offline_opt = A[row_ind, col_ind].sum()
                _, greed_value = dp.greedy(A, coin_flips)
                learned_vals.append(value / offline_opt)
                greedy_vals.append(greed_value / offline_opt)
            
            learned_mean = np.mean(learned_vals); greedy_mean = np.mean(greedy_vals)
            learned_std = np.std(learned_vals, ddof=1); greedy_std = np.std(greedy_vals, ddof=1)
            print(f"Learned competitive ratio: {learned_mean} ± {2 * learned_std / np.sqrt(num_trials)}")
            print(f"Greedy competitive ratio: {greedy_mean} ± {2 * greedy_std / np.sqrt(num_trials)}")
            print()

test_model(trained_model, num_trials, node_configs, graph_configs)

In [ ]:
num_trials = 250
node_configs = [(24, 6), (28, 7), (10, 10), (11, 11), (20, 5), (24, 6)]
graph_configs = [
    {
        'graph_type': 'ER',
        'p': 0.75,
        'weighted': True
    },
    {
        'graph_type': 'BA',
        'ba_param': 4,
        'weighted': True
    },
    {
        'graph_type': 'GEOM',
        'threshold': 0.2,
        'scaling': 1 / np.sqrt(2)
    }
]
for config in graph_configs:    
    for (m, n) in node_configs:
        print(f"online: {m}, offline: {n}, config: {config}")
        ratios = []
        for _ in range(num_trials):
            A = gg.sample_bipartite_graph(m, n, **config)
            p = [0.8 for _ in range(m)]
            coin_flips = [np.random.binomial(1, _p) for _p in p]
            cache = dp.cache_stochastic_opt(A, p)
            _, stoch_opt = dp.stochastic_opt(A, coin_flips, cache)
            _, offline_opt = dp.offline_opt(A, coin_flips)
            ratios.append(stoch_opt / offline_opt)
        mean_ratio = np.mean(ratios);
        std_ratio = np.std(ratios, ddof=1)
        print(f"Stoch opt competitive ratio: {mean_ratio} ± {2 * std_ratio / np.sqrt(num_trials)}")
        print()

            

In [ ]:
m = 10; n = 6; num_trials = 100

config = {
    'graph_type': 'ER',
    'p': 0.75,
    'weighted': True
}
def error_code(chosen_index, pred_index, length):
    if chosen_index == pred_index and chosen_index == length - 1:
        return "Correct skip"
    elif chosen_index == pred_index:
        return "Correct no skip"
    elif chosen_index == length - 1:
        return "Incorrect no skip"
    elif pred_index == length - 1:
        return "Incorrect skip"
    else:
        return "Incorrect choice"
    
record = ([[] for _ in range(m)], [[] for _ in range(m)], [[] for _ in range(m)])
greedy_vals = []
learned_vals = []
for i in range(num_trials):
    A = gg.sample_bipartite_graph(m, n, **config)
    p = [1 for _ in range(m)]
    cache = dp.cache_stochastic_opt(A, p)
    coin_flips = [np.random.binomial(1, _p) for _p in p]
    all_nodes = np.arange(n + m + 1)
    offline_nodes = frozenset(np.arange(n))
    matching = []
    value = 0
    for t in range(m):
        if coin_flips[t]:
            input = gg._to_pyg_test(A, p, offline_nodes, t)
            pred = trained_model(input.x, input.edge_index, input.edge_attr)
            
            hints = dp.one_step_stochastic_opt(A, offline_nodes, t, cache)
            hints = np.max(hints) - hints
            print(t)
            print(pred[input.neighbors].squeeze().detach().numpy())
            print(hints)
            print()
            chosen_index = np.argmin(pred[input.neighbors].detach().numpy())
            opt_index = np.argmin(hints)
            choice = all_nodes[input.neighbors][chosen_index]
            correct = (chosen_index == opt_index)
            reduction = hints[opt_index] - hints[chosen_index]
            record[0][t].append(error_code(opt_index, chosen_index, len(hints)))
            record[1][t].append(reduction)
            record[2][t].append(np.sum(input.neighbors.detach().numpy()))
            if choice < n:
                matching.append((t, choice))
                value += A[t, choice]
        
            offline_nodes = diff(offline_nodes, choice)

   
    opt_matching, opt_value = dp.stochastic_opt(A, coin_flips, cache)
    _, greed_value = dp.greedy(A, coin_flips)
    learned_vals.append(value / opt_value)
    greedy_vals.append(greed_value / opt_value)

In [ ]:
#Need to predict skip more often?
import matplotlib.pyplot as plt
for t in range(m):
  labels, vals =  np.unique(record[0][t], return_counts=True)
  plt.bar(labels, vals)
  plt.xticks(rotation=45)
  plt.show()

In [ ]:
import matplotlib.pyplot as plt

for i in range(len(record[0])):
    print(f"Node: {i}, average reduction: {np.mean(record[1][i])}")
    print(f"Node: {i}, average neighbors: {np.mean(record[2][i])}")
    plt.hist(record[1][i], bins=10)
    plt.show()

In [ ]:
matching

In [ ]:
opt_matching

In [ ]:
value, opt_value

In [ ]:
greed_value

In [ ]:
m = 10; n = 6; num_trials = 1000

# config = {
#     'graph_type': 'ER',
#     'p': 0.75,
#     'weighted': True
# }
config = {
    'graph_type': 'GEOM',
    'threshold': 0.2,
    'scaling': 1 / np.sqrt(2)
}

def _mask(offline_nodes, adj):
    return [adj[i] for i in range(n) if i in offline_nodes and adj[i] > 0]


record = ([[] for _ in range(m)], [[] for _ in range(m)], [[] for _ in range(m)])
greedy_vals = []
learned_vals = []
for i in range(num_trials):
    A = gg.sample_bipartite_graph(m, n, **config)
    p = [1 for _ in range(m)]
    cache = dp.cache_stochastic_opt(A, p)
    coin_flips = [np.random.binomial(1, _p) for _p in p]
    all_nodes = np.arange(n + m + 1)
    offline_nodes = frozenset(np.arange(n))
    matching = []
    value = 0
    for t in range(m):
        if coin_flips[t]:
            input = gg._to_pyg_test(A, p, offline_nodes, t)
            pred = trained_model(input.x, input.edge_index, input.edge_attr) \
              [input.neighbors].detach().numpy()
            masked_adj = _mask(offline_nodes, A[t, :])
            hints = dp.one_step_stochastic_opt(A, offline_nodes, t, cache)
            learned_index = np.argmax(pred)
            try:
              greedy_index = np.argmax(masked_adj)
            except:
              greedy_index = learned_index
            
            opt_index = np.argmax(hints)
            if pred[learned_index] - pred[greedy_index] > 0.05:
                chosen_index = learned_index
            else:
                chosen_index = greedy_index
            
            choice = all_nodes[input.neighbors][chosen_index]
            correct = (chosen_index == opt_index)
            reduction = hints[opt_index] - hints[chosen_index]
            record[0][t].append(correct)
            record[1][t].append(reduction)
            record[2][t].append(np.sum(input.neighbors.detach().numpy()))
            if choice < n:
                matching.append((t, choice))
                value += A[t, choice]
        
            offline_nodes = diff(offline_nodes, choice)

   
    opt_matching, opt_value = dp.stochastic_opt(A, coin_flips, cache)
    _, greed_value = dp.greedy(A, coin_flips)
    learned_vals.append(value / opt_value)
    greedy_vals.append(greed_value / opt_value)

In [ ]:
learned_mean = np.mean(learned_vals); greedy_mean = np.mean(greedy_vals)
learned_std = np.std(learned_vals, ddof=1); greedy_std = np.std(greedy_vals, ddof=1)
print(f"Learned competitive ratio: {learned_mean} ± {2 * learned_std / np.sqrt(num_trials)}")
print(f"Greedy competitive ratio: {greedy_mean} ± {2 * greedy_std / np.sqrt(num_trials)}")

for i in range(len(record[0])):
    print(f"Node: {i}, proportion correct: {np.mean(record[0][i])}")

In [ ]:
for i in range(len(record[0])):
    print(f"Node: {i}, average reduction: {np.mean(record[1][i])}")
    print(f"Node: {i}, average neighbors: {np.mean(record[2][i])}")
    plt.hist(record[1][i], bins=25)
    plt.show()